In [ ]:
%pip install llama-index-agent-openai
%pip install llama-index-llms-openai

# Install required libraries
%pip install llama-index




Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Import necessary libraries
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.llms.ollama import Ollama
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent

In [9]:
# Configure Ollama LLM
ollama_llm = Ollama(
    model="llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding


In [15]:
march_2022 = SimpleDirectoryReader(input_files=["./data/uber_10q_march_2022.pdf"]).load_data()
june_2022 = SimpleDirectoryReader(input_files=["./data/uber_10q_june_2022.pdf"]).load_data()
sept_2022 = SimpleDirectoryReader(input_files=["./data/uber_10q_sept_2022.pdf"]).load_data()

In [18]:
import os


def get_tool(name, full_name, documents=None):
    if not os.path.exists(f"./data/{name}"):
        # build vector index
        vector_index = VectorStoreIndex.from_documents(documents)
        vector_index.storage_context.persist(persist_dir=f"./data/{name}")
    else:
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{name}"),
        )
    query_engine = vector_index.as_query_engine(similarity_top_k=3, llm=ollama_llm)
    query_engine_tool = QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=name,
            description=(
                "Provides information about Uber quarterly financials ending"
                f" {full_name}"
            ),
        ),
    )
    return query_engine_tool

In [19]:
march_tool = get_tool("march_2022", "March 2022", documents=march_2022)
june_tool = get_tool("june_2022", "June 2022", documents=june_2022)
sept_tool = get_tool("sept_2022", "September 2022", documents=sept_2022)

In [20]:
query_engine_tools = [march_tool, june_tool, sept_tool]

In [21]:
from llama_index.core.agent import AgentRunner, ReActAgent
from llama_index.agent.openai import OpenAIAgentWorker, OpenAIAgent
from llama_index.agent.openai import OpenAIAgentWorker

# openai_step_engine = OpenAIAgentWorker.from_tools(
#     query_engine_tools, llm=llm, verbose=True
# )
# agent = AgentRunner(openai_step_engine)
# # alternative
# agent = OpenAIAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

#agent_llm = OpenAI(model="gpt-3.5-turbo")
# agent_llm = OpenAI(model="gpt-4-1106-preview")

agent = ReActAgent.from_tools(
    query_engine_tools, llm=ollama_llm, verbose=True, max_iterations=20
)

In [31]:
response = agent.chat("Analyze the changes in R&D expenditures and revenue")


> Running step 34c53053-4e9a-463c-b930-7c8e222cb42f. Step input: Analyze the changes in R&D expenditures and revenue
Thought: I can analyze the changes in R&D expenditures and revenue using the provided tools.
Action: march_2022
Action Input: {'input': 'R&D expenditures and revenue analysis for Q3 2021 to Q2 2022', 'date': ['Q3 2021', 'Q2 2022']}
Observation: Research and development (R&D) expenses increased by $72 million, or 14%, primarily due to an increase in consumer advertising expenses. 

Gross Bookings for Mobility increased from $5,905 million in Q3 2020 to $6,789 million in Q4 2020, then to $6,773 million in Q1 2021, and finally to $8,640 million in Q2 2021. This represents a year-over-year increase of 85%. 

Gross Bookings for Delivery increased from $8,550 million in Q3 2020 to $10,050 million in Q4 2020, then to $12,461 million in Q1 2021, and finally to $12,828 million in Q2 2021. This represents a year-over-year increase of 51%. 

Gross Bookings for Freight increased fro

In [32]:
print(str(response))


The analysis of changes in R&D expenditures and revenue for Uber shows a significant increase in R&D spending from Q3 2021 to Q2 2022, with a year-over-year increase of 54%. However, despite this increase, R&D expenses still account for only 9% of overall revenue.


In [33]:
# start task
task = agent.create_task("Analyze the changes in R&D expenditures and revenue")

In [34]:
step_output = agent.run_step(task.task_id)

> Running step c2536e76-8548-41dc-98f9-8cada11b7c53. Step input: Analyze the changes in R&D expenditures and revenue
Observation: Error: Could not parse output. Please follow the thought-action-input format. Try again.


In [35]:
step_output = agent.run_step(task.task_id)

> Running step d561a314-1022-478e-a711-edc3017d4692. Step input: None
Thought: I need to use a tool to help me analyze the changes in R&D expenditures and revenue for Uber.
Action: march_2022
Action Input: {'input': 'Uber Q3 2021 vs Q2 2022 R&D expenditures and revenue analysis'}
Observation: According to the provided financial statements, Research and Development (R&D) expenditures for Q3 2021 were $515 million, while for Q2 2022, they were $587 million. This represents an increase of $72 million or 14% from Q3 2021 to Q2 2022.

As for revenue, the total revenue for Q3 2021 was $2,903 million, and for Q2 2022, it was $6,854 million. This represents a significant increase of $4,951 million or 172% from Q3 2021 to Q2 2022. The growth in revenue can be attributed to the increasing number of Mobility and Delivery orders on the platform, as well as the expansion of new mobility services.

It's worth noting that the improvement in Adjusted EBITDA from a loss of $359 million in Q3 2021 to $1

In [36]:
step_output = agent.run_step(task.task_id, input="What about June?")

> Running step b048d51b-5bee-4a7c-94c8-03eb72298fa3. Step input: What about June?
Added user message to memory: What about June?
Thought: I need to use a tool to help me analyze the changes in R&D expenditures and revenue for Uber from Q2 2022.
Action: june_2022
Action Input: {'input': 'Uber Q2 2022 vs Q3 2021 R&D expenditures and revenue analysis'}
Observation: Research and development expenditures increased by $216 million, or 44%, from $488 million in Q2 2021 to $704 million in Q2 2022. 

Revenue from research and development expenditures as a percentage of total revenue decreased from 6.3% in Q2 2021 to 8.7% in Q2 2022.


In [37]:
step_output = agent.run_step(task.task_id, input="What about September?")

> Running step 26f38937-b518-4e3d-9ca9-b3b3218311f3. Step input: What about September?
Added user message to memory: What about September?
Thought: I need to use a tool to help me analyze the changes in R&D expenditures and revenue for Uber from Q3 2022.
Action: sept_2022
Action Input: {'input': 'Uber Q3 2022 vs Q2 2022 R&D expenditures and revenue analysis'}
Observation: Research and development expenditures increased by $277 million from $493 million in Q2 2022 to $760 million in Q3 2022. This represents a growth rate of 51% compared to the same period last year.

Freight revenue, which is part of the Research and Development segment, grew by $4.4 billion from $1,431 million in Q2 2022 to $1,878 million in Q3 2022. Freight Adjusted EBITDA improved by $113 million, driven primarily by the growth in Freight revenue.

The increase in R&D expenditures is largely attributed to the growth in Freight revenue, as well as investments in other areas of the business. However, it's worth noting 

In [38]:
step_output = agent.run_step(task.task_id)

> Running step cc6554e9-2bb6-42fb-816b-e5555aecb028. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The analysis shows that Uber's Research and Development expenditures have been increasing steadily over the past few quarters, with significant growth rates of 14% from Q3 2021 to Q2 2022 and 51% from Q2 2022 to Q3 2022. This increase is largely attributed to the growth in Freight revenue, as well as investments in other areas of the business.


In [39]:
response = agent.finalize_response(task.task_id)

In [40]:
print(str(response))

The analysis shows that Uber's Research and Development expenditures have been increasing steadily over the past few quarters, with significant growth rates of 14% from Q3 2021 to Q2 2022 and 51% from Q2 2022 to Q3 2022. This increase is largely attributed to the growth in Freight revenue, as well as investments in other areas of the business.


In [ ]:
# agent_llm = OpenAI(model="gpt-3.5-turbo")
# agent_llm = OpenAI(model="gpt-4-1106-preview")

agent = ReActAgent.from_tools(
    query_engine_tools, llm=ollama_llm, verbose=True, max_iterations=20
)